<h1>Cecil Gilliard Crypto Trading bot</h1>

<h3>This is code to produce a BTC-USD ticker from Coinbase Sanbox API</h3>

<h4>My Youtube link: https://www.youtube.com/watch?v=JNWjZ6bOAk4</h4>

In [4]:
import pandas as pd
import cbpro
import time

# credit to this dev for the ticker
# https://www.youtube.com/watch?v=a74pQbHgdXw

class TextWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url = 'wss://ws-feed-public.sandbox.pro.coinbase.com'
        self.message_count = 0
    
    def on_message(self,msg):
        self.message_count += 1
        msg_type = msg.get('type',None)
        if msg_type == 'ticker':
            time_val   = msg.get('time',('-'*10))
            price_val  = msg.get('price',None)
            if price_val is not None:
                price_val = float(price_val)
            product_id = msg.get('product_id',None)
            
            print(f"{time_val:30} \
                {price_val:.3f} \
                {msg_type}: \
                {product_id}")
    
    def on_close(self):
        print(f"<---Websocket connection closed--->\n\tTotal messages: {self.message_count}")

In [5]:
# used to start the ticker
stream = TextWebsocketClient(products=['BTC-USD'],channels=['ticker'])
stream.start()

2021-12-06T17:01:21.883966Z                    49400.000                 ticker:                 BTC-USD
2021-12-06T17:01:22.788104Z                    49522.010                 ticker:                 BTC-USD
2021-12-06T17:01:22.987637Z                    50000.000                 ticker:                 BTC-USD
2021-12-06T17:01:23.632466Z                    50000.000                 ticker:                 BTC-USD
2021-12-06T17:01:24.044885Z                    50000.000                 ticker:                 BTC-USD
2021-12-06T17:01:24.302470Z                    50000.000                 ticker:                 BTC-USD
2021-12-06T17:01:24.302470Z                    49600.000                 ticker:                 BTC-USD
2021-12-06T17:01:24.488460Z                    57564.000                 ticker:                 BTC-USD
2021-12-06T17:01:24.643662Z                    49600.000                 ticker:                 BTC-USD
2021-12-06T17:01:24.828967Z                    49624.11

In [6]:
# used to stop the ticker
stream.close()

2021-12-06T17:01:26.862098Z                    47800.000                 ticker:                 BTC-USD
2021-12-06T17:01:27.010472Z                    48644.610                 ticker:                 BTC-USD
<---Websocket connection closed--->
	Total messages: 42


<h3>This is a simple program used for buying and selling BTC manually through the Coinbase Sandbox API</h3>

In [7]:
import cbpro
import time
#  created my api key and saved secret, key, and passphrase on seperate file for security
from cb_authy import (api_secret,
                      api_key,
                      api_pass) 

# Create authenticatedClient instance
# this allows for buys/sells deposits/withdrawels from account
# https://pypi.org/project/cbpro/

url='https://api-public.sandbox.pro.coinbase.com'

client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass,
    api_url=url
)
pub_client = cbpro.PublicClient()

# the method one would use to deposit into account
# I've already deposited so commented out
# payment_methods = client.get_payment_methods()

# for method in payment_methods:
#     currency  = method.get('currency',None)
#     if currency.upper() == 'USD':
#         method_id = method.get('id',None) 
#     elif currency is None:
#         continue
        
# client.deposit(
#     amount=10000,
#     currency=currency,
#     payment_method_id=method_id
# )


# set spot price to variable 
price = float(client.get_product_ticker(product_id='BTC-USD')['price'])

buy_sell = input("Would you like to buy or sell BTC? ").lower()


if (buy_sell == "buy"):
    print(f'The current price of BTC-USD is {price} USD.')
    buy_price = input("At what price in USD would you like to buy BTC? ")
    if (float(buy_price) >= price):
        buy_amt = input("How much BTC would you like to purchase? 'USD': ")
        print("Buying BTC!!!")
        print(client.buy(funds=buy_amt, order_type="market", product_id="BTC-USD"))
    elif (float(buy_price) <= price):
        buy_sze = input("How much BTC would you like to purchase? 'BTC': ")
        print("Setting limit order!!")
        print(client.buy(price=buy_price, size=buy_sze, order_type="limit", product_id="BTC-USD"))
        
elif (buy_sell == "sell"):
    print(f'The current price of BTC-USD is {price} USD.')
    sell_price = input("At what price in USD would you like to sell BTC? ")
    if (float(sell_price) <= price):
        sell_amt = input("How much BTC would you like to sell? USD: ")
        print("Selling BTC!!!")
        print(client.sell(funds=sell_amt, order_type="market", product_id="BTC-USD"))
    elif (float(sell_price) >= price):
        sell_sze = input("How much BTC would you like to sell? 'BTC': ")
        print("Setting limit order!!")
        print(client.sell(price=buy_price, size=sell_sze, order_type="limit", product_id="BTC-USD"))

else: 
    print("Error: Must select 'buy or sell'.")

    
cancel_orders = input("Would you like to cancel all BTC-USD limit orders? 'Y/N'").upper()


if (cancel_orders == "Y"):
    u_sure = input("Are you sure you would like to cancel all BTC-USD limit orders?! 'Y/N'")
    if (u_sure == 'Y'):
        print(client.cancel_all(product_id="BTC-USD"))
        print("All orders cancelled!!")
else:
    pass
        


Would you like to buy or sell BTC?buy
The current price of BTC-USD is 49741.2 USD.
At what price in USD would you like to buy BTC? 49741.2
How much BTC would you like to purchase? 'USD': 100
Buying BTC!!!
{'id': '534d9ea9-8011-4318-8a59-89a3527af864', 'product_id': 'BTC-USD', 'side': 'buy', 'stp': 'dc', 'funds': '99.50248756', 'specified_funds': '100', 'type': 'market', 'post_only': False, 'created_at': '2021-12-06T17:02:54.075147Z', 'fill_fees': '0', 'filled_size': '0', 'executed_value': '0', 'status': 'pending', 'settled': False}
Would you like to cancel all BTC-USD limit orders? 'Y/N'N


<h3>Trading bot for BTC-USD using the Coinbase Sandbox API</h3>

In [1]:
import cbpro
import time
#  created my api key and saved secret, key, and passphrase on seperate file for security
from cb_authy import (api_secret,
                      api_key,
                      api_pass) 

# Create authenticatedClient instance
# this allows for buys/sells deposits/withdrawels from account
# https://pypi.org/project/cbpro/

url='https://api-public.sandbox.pro.coinbase.com'

client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass,
    api_url=url
)
pub_client = cbpro.PublicClient()
# set spot price to variable 
price = float(client.get_product_ticker(product_id='BTC-USD')['price'])
# get the opening price of BTC
stats = client.get_product_24hr_stats('BTC-USD')
open_price = float(stats['open'])

# function to decide percentage change between two values
# https://stackoverflow.com/questions/30926840/how-to-check-change-between-two-values-in-percent
def percentage_change(current, previous):
    """returns percent difference between 2 numbers"""
    if previous != 0 :
        return round(float(current - previous) / abs(previous) * 100, 2)
    else:
        return "undefined"

buy_amt = 100
sell_amt = 500

# bot to buy or sell btc based on percentage change between opening and current price
# obviously this is a horrible strategy; but it shows the endless ways a program can 
# implement strategies to buy/sell cryptocurrency without the user's manual input
while (True):   
    # aribitrary percentenge change numbers, can make whatever you'd like
    if percentage_change(price, open_price) >= -2:
        print("BUY BUY BUY!!!")
        # actual code to buy
#         print(client.buy(funds=buy_amt, order_type="market", product_id="BTC-USD"))
        # i have the breaks in here just to stop the infinite loop
#         break
    elif percentage_change(price, open_price) >= 2:
        print("SELL SELL SELL!!!")
        # actual code to sell
#         print(client.sell(funds=sell_amt, order_type="market", product_id="BTC-USD"))
#         break
    else:
        print("C'mon, do something BTC!!!")
#         break
    # timer to set bot to trading timeframe in seconds
    time.sleep(5)
    
# print(open_price)
# print(price)

C'mon, do something BTC!!!
C'mon, do something BTC!!!
C'mon, do something BTC!!!


KeyboardInterrupt: 